#### Data Gathering & Visualization

In [1]:
from __future__ import with_statement
from AnalyticsClient import AnalyticsClient
import pandas as pd
import plotly.express as px
import warnings
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
warnings.filterwarnings('ignore')

class Config:

    CLIENTID = "1000.DQ32DWGNGDO7CV0V1S1CB3QFRAI72K";
    CLIENTSECRET = "92dfbbbe8c2743295e9331286d90da900375b2b66c";
    REFRESHTOKEN = "1000.0cd324af15278b51d3fc85ed80ca5c04.7f4492eb09c6ae494a728cd9213b53ce";

    ORGID = "60006357703";
    VIEWID="174857000094424767";
    WORKSPACEID = "174857000004732522";
    
class sample:
    ac = AnalyticsClient(Config.CLIENTID, Config.CLIENTSECRET, Config.REFRESHTOKEN)

    def export_data(self, ac, view_ids):
        response_format = "csv"
        file_path_template = "Impression_{}.csv"
        bulk = ac.get_bulk_instance(Config.ORGID, Config.WORKSPACEID)
        for view_id in view_ids:
            file_path = file_path_template.format(view_id)
            bulk.export_data(view_id, response_format, file_path)

try:
    obj = sample()
    view_ids = ["174857000094424767"]  # Replace with your list of view IDs
    obj.export_data(obj.ac, view_ids)

except Exception as e:
    print(str(e))
    

df=pd.read_csv('Impression_174857000094424767.csv')
df['Dates']=pd.to_datetime(df['Dates'])
df= df.sort_values(by='Dates', ascending=True)
df['Buyer GMV']=df['Buyer GMV'].astype(int)

In [2]:
df.head()

,Dates,BuyerId,BuyerName,BuyerPhone,SellerId,SellerName,SellerPhone,Recency Bucket,Buyer GMV
0,2023-06-01,1490270381,s.v.traders,7899819755,1490414492,YES TRADERS,8700630916,Above 90 Days,3951
27,2023-06-01,1491150419,durva footwear,7387029148,1490414492,YES TRADERS,8700630916,Above 90 Days,3786
26,2023-06-01,1491146328,Shri mahasu empire,9459729994,1490482693,MAHESHWARI GARMENT,8502830016,Above 90 Days,6494
25,2023-06-01,1491120471,Anamika store,9707738692,1490482693,MAHESHWARI GARMENT,8502830016,Above 90 Days,6760
24,2023-06-01,1491093648,Rsr.mega.mart,9956649171,1490482693,MAHESHWARI GARMENT,8502830016,16-30 Days,4060


In [3]:

df['Dates'] = pd.to_datetime(df['Dates'])
df = df.sort_values('Dates')
days_ago = pd.Timestamp.now() - pd.DateOffset(days=10)

# Filter DataFrame to get data for the last 5 days
last_n_days = df[df['Dates'] >= days_ago]

# Add a sequential order column
last_n_days['Order'] = range(1, len(last_n_days) + 1)

# Plot the sequential curve graph using Plotly
fig = px.line(last_n_days, x='Order', y='Buyer GMV', color='Dates',
              title='GMV for Last 5 Days',
              labels={'Order': 'Sequence', 'Buyer GMV': 'GMV', 'Dates': 'Date'},
              hover_data={'BuyerId':True},
              template='plotly_dark')
fig.show()


#### Data Transformation

In [4]:
daily_stats_seller = df.groupby(['Dates', 'SellerId'])['Buyer GMV'].agg(['mean', 'std']).reset_index()
df = df.merge(daily_stats_seller, on=['Dates', 'SellerId'], how='left', suffixes=('', '_daily'))
df['GMV'] = (df['Buyer GMV'] - df['mean']) / df['std']
df = df.dropna()
df.head()


,Dates,BuyerId,BuyerName,BuyerPhone,SellerId,SellerName,SellerPhone,Recency Bucket,Buyer GMV,mean,std,GMV
0,2023-06-01,1490270381,s.v.traders,7899819755,1490414492,YES TRADERS,8700630916,Above 90 Days,3951,3700.3125,2034.826503,0.123198
1,2023-06-01,1490485595,Progressive marketing,9567139155,1490414492,YES TRADERS,8700630916,Above 90 Days,2190,3700.3125,2034.826503,-0.742232
2,2023-06-01,1490706971,Puja store,7859077362,1490482693,MAHESHWARI GARMENT,8502830016,Above 90 Days,3354,4248.0000,1796.468295,-0.497643
3,2023-06-01,1490696149,Tei 100dawr,8794786070,1490414492,YES TRADERS,8700630916,Above 90 Days,4498,3700.3125,2034.826503,0.392017
4,2023-06-01,1490629707,Shakir footwear,8638848970,1490414492,YES TRADERS,8700630916,61-90 Days,2680,3700.3125,2034.826503,-0.501425


In [5]:
def cluster(x):
    if x>0:
        return 1
    else :
        return 0
df['Cluster']=df['GMV'].apply(cluster)

In [6]:
df

,Dates,BuyerId,BuyerName,BuyerPhone,SellerId,SellerName,SellerPhone,Recency Bucket,Buyer GMV,mean,std,GMV,Cluster
0,2023-06-01,1490270381,s.v.traders,7899819755,1490414492,YES TRADERS,8700630916,Above 90 Days,3951,3700.3125,2034.826503,0.123198,1
1,2023-06-01,1490485595,Progressive marketing,9567139155,1490414492,YES TRADERS,8700630916,Above 90 Days,2190,3700.3125,2034.826503,-0.742232,0
2,2023-06-01,1490706971,Puja store,7859077362,1490482693,MAHESHWARI GARMENT,8502830016,Above 90 Days,3354,4248.0000,1796.468295,-0.497643,0
3,2023-06-01,1490696149,Tei 100dawr,8794786070,1490414492,YES TRADERS,8700630916,Above 90 Days,4498,3700.3125,2034.826503,0.392017,1
4,2023-06-01,1490629707,Shakir footwear,8638848970,1490414492,YES TRADERS,8700630916,61-90 Days,2680,3700.3125,2034.826503,-0.501425,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9739,2024-06-02,1491174324,Tarique shoe house,7872245927,1490414492,YES TRADERS,8700630916,0-7 Days,3262,4232.1000,860.828083,-1.126938,0
9740,2024-06-02,1491178332,Anaya Store,8822015146,1490414492,YES TRADERS,8700630916,0-7 Days,4249,4232.1000,860.828083,0.019632,1
9741,2024-06-02,1490510342,Aiwa footwear,9142336633,1490414492,YES TRADERS,8700630916,0-7 Days,4910,4232.1000,860.828083,0.787498,1
9742,2024-06-03,1490386857,choice shoe,9007496889,1490414492,YES TRADERS,8700630916,0-7 Days,4033,3810.0000,315.369624,0.707107,1


In [7]:
pivot_df=df.groupby(['SellerId','BuyerId'])['Cluster'].sum().reset_index()
pivot_df=pivot_df.sort_values(by='Cluster',ascending=False)

pivot_df

,SellerId,BuyerId,Cluster
2503,1490414492,1491076443,19
3579,1490482693,1491061923,10
804,1490414492,1490351466,10
2303,1490414492,1490979809,9
1755,1490414492,1490696149,9
...,...,...,...
2483,1490414492,1491068330,0
2485,1490414492,1491069424,0
2486,1490414492,1491069431,0
2488,1490414492,1491069843,0


In [8]:
pivot_df['Rank']=pivot_df.groupby('SellerId')['Cluster'].rank(method='first',ascending=False).astype(int)
def buyer_priority(x):
    if x>0 :
        return 'High Priority'
    else:
        return 'Low Priority'
pivot_df['Priority']=pivot_df['Cluster'].apply(buyer_priority)
pivot_df.head(30)

,SellerId,BuyerId,Cluster,Rank,Priority
2503,1490414492,1491076443,19,1,High Priority
3579,1490482693,1491061923,10,1,High Priority
804,1490414492,1490351466,10,2,High Priority
2303,1490414492,1490979809,9,3,High Priority
1755,1490414492,1490696149,9,4,High Priority
1430,1490414492,1490525727,9,5,High Priority
2449,1490414492,1491048831,8,6,High Priority
3771,1490482693,1491148662,8,2,High Priority
5214,1490485625,1490734058,8,1,High Priority
539,1490414492,1479646198,8,7,High Priority


In [9]:
final_df=pivot_df[pivot_df['Priority']=='High Priority']

In [10]:
final_df.head()

,SellerId,BuyerId,Cluster,Rank,Priority
2503,1490414492,1491076443,19,1,High Priority
3579,1490482693,1491061923,10,1,High Priority
804,1490414492,1490351466,10,2,High Priority
2303,1490414492,1490979809,9,3,High Priority
1755,1490414492,1490696149,9,4,High Priority


In [11]:
final_df['Key']=final_df['SellerId'].astype(str)+'_'+final_df['BuyerId'].astype(str)
final_df

,SellerId,BuyerId,Cluster,Rank,Priority,Key
2503,1490414492,1491076443,19,1,High Priority,1490414492_1491076443
3579,1490482693,1491061923,10,1,High Priority,1490482693_1491061923
804,1490414492,1490351466,10,2,High Priority,1490414492_1490351466
2303,1490414492,1490979809,9,3,High Priority,1490414492_1490979809
1755,1490414492,1490696149,9,4,High Priority,1490414492_1490696149
...,...,...,...,...,...,...
834,1490414492,1490359929,1,954,High Priority,1490414492_1490359929
980,1490414492,1490390332,1,955,High Priority,1490414492_1490390332
866,1490414492,1490366326,1,956,High Priority,1490414492_1490366326
3414,1490482693,1490947359,1,461,High Priority,1490482693_1490947359


In [12]:
df['Key']=df['SellerId'].astype(str)+'_'+df['BuyerId'].astype(str)
df

,Dates,BuyerId,BuyerName,BuyerPhone,SellerId,SellerName,SellerPhone,Recency Bucket,Buyer GMV,mean,std,GMV,Cluster,Key
0,2023-06-01,1490270381,s.v.traders,7899819755,1490414492,YES TRADERS,8700630916,Above 90 Days,3951,3700.3125,2034.826503,0.123198,1,1490414492_1490270381
1,2023-06-01,1490485595,Progressive marketing,9567139155,1490414492,YES TRADERS,8700630916,Above 90 Days,2190,3700.3125,2034.826503,-0.742232,0,1490414492_1490485595
2,2023-06-01,1490706971,Puja store,7859077362,1490482693,MAHESHWARI GARMENT,8502830016,Above 90 Days,3354,4248.0000,1796.468295,-0.497643,0,1490482693_1490706971
3,2023-06-01,1490696149,Tei 100dawr,8794786070,1490414492,YES TRADERS,8700630916,Above 90 Days,4498,3700.3125,2034.826503,0.392017,1,1490414492_1490696149
4,2023-06-01,1490629707,Shakir footwear,8638848970,1490414492,YES TRADERS,8700630916,61-90 Days,2680,3700.3125,2034.826503,-0.501425,0,1490414492_1490629707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9739,2024-06-02,1491174324,Tarique shoe house,7872245927,1490414492,YES TRADERS,8700630916,0-7 Days,3262,4232.1000,860.828083,-1.126938,0,1490414492_1491174324
9740,2024-06-02,1491178332,Anaya Store,8822015146,1490414492,YES TRADERS,8700630916,0-7 Days,4249,4232.1000,860.828083,0.019632,1,1490414492_1491178332
9741,2024-06-02,1490510342,Aiwa footwear,9142336633,1490414492,YES TRADERS,8700630916,0-7 Days,4910,4232.1000,860.828083,0.787498,1,1490414492_1490510342
9742,2024-06-03,1490386857,choice shoe,9007496889,1490414492,YES TRADERS,8700630916,0-7 Days,4033,3810.0000,315.369624,0.707107,1,1490414492_1490386857


In [13]:
df_new=df[['Key','Recency Bucket','SellerName','BuyerName','SellerPhone','BuyerPhone']]

In [14]:
final_df=final_df.merge(df_new,how='inner',on='Key')
final_df=final_df.drop_duplicates()
final_df.head()

,SellerId,BuyerId,Cluster,Rank,Priority,Key,Recency Bucket,SellerName,BuyerName,SellerPhone,BuyerPhone
0,1490414492,1491076443,19,1,High Priority,1490414492_1491076443,8-15 Days,YES TRADERS,Suzuki footwear,8700630916,8870981853
36,1490482693,1491061923,10,1,High Priority,1490482693_1491061923,16-30 Days,MAHESHWARI GARMENT,dress code,8502830016,8976147275
56,1490414492,1490351466,10,2,High Priority,1490414492_1490351466,31-60 Days,YES TRADERS,Kids collection,8700630916,9334408279
72,1490414492,1490979809,9,3,High Priority,1490414492_1490979809,8-15 Days,YES TRADERS,Sadham Foot Wear,8700630916,9080437790
86,1490414492,1490696149,9,4,High Priority,1490414492_1490696149,Above 90 Days,YES TRADERS,Tei 100dawr,8700630916,8794786070


In [15]:
final_df=final_df[['SellerId', 'SellerName', 'BuyerId','BuyerName','BuyerPhone',  'Recency Bucket']]
final_df

,SellerId,SellerName,BuyerId,BuyerName,BuyerPhone,Recency Bucket
0,1490414492,YES TRADERS,1491076443,Suzuki footwear,8870981853,8-15 Days
36,1490482693,MAHESHWARI GARMENT,1491061923,dress code,8976147275,16-30 Days
56,1490414492,YES TRADERS,1490351466,Kids collection,9334408279,31-60 Days
72,1490414492,YES TRADERS,1490979809,Sadham Foot Wear,9080437790,8-15 Days
86,1490414492,YES TRADERS,1490696149,Tei 100dawr,8794786070,Above 90 Days
...,...,...,...,...,...,...
5123,1490414492,YES TRADERS,1490359929,long life footwear,9656677560,0-7 Days
5126,1490414492,YES TRADERS,1490390332,Tanisha Shoe Stores,9609458847,16-30 Days
5132,1490414492,YES TRADERS,1490366326,Hamid footwear,9608349584,Above 90 Days
5133,1490482693,MAHESHWARI GARMENT,1490947359,G9 collection,8619922492,Above 90 Days


In [16]:
final_df.to_excel('1_May_Buyer_Target.xlsx',index=False)

In [17]:
final_df.shape

(2421, 6)

In [18]:
final_df

,SellerId,SellerName,BuyerId,BuyerName,BuyerPhone,Recency Bucket
0,1490414492,YES TRADERS,1491076443,Suzuki footwear,8870981853,8-15 Days
36,1490482693,MAHESHWARI GARMENT,1491061923,dress code,8976147275,16-30 Days
56,1490414492,YES TRADERS,1490351466,Kids collection,9334408279,31-60 Days
72,1490414492,YES TRADERS,1490979809,Sadham Foot Wear,9080437790,8-15 Days
86,1490414492,YES TRADERS,1490696149,Tei 100dawr,8794786070,Above 90 Days
...,...,...,...,...,...,...
5123,1490414492,YES TRADERS,1490359929,long life footwear,9656677560,0-7 Days
5126,1490414492,YES TRADERS,1490390332,Tanisha Shoe Stores,9609458847,16-30 Days
5132,1490414492,YES TRADERS,1490366326,Hamid footwear,9608349584,Above 90 Days
5133,1490482693,MAHESHWARI GARMENT,1490947359,G9 collection,8619922492,Above 90 Days


In [19]:
from google.oauth2 import service_account
import gspread
from datetime import datetime

# Set your JSON key file path
json_key_path = 'Supply_cred.json'

# Load credentials from the JSON key file
creds = service_account.Credentials.from_service_account_file(json_key_path, scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

# Authorize the client using the credentials
client = gspread.authorize(creds)

# Open the Google Sheets document by title
source_sheet_title = 'DOD Details - Whatsapp bulk'
source_sheet = client.open(source_sheet_title)

# Select a specific worksheet within the source document
source_worksheet = source_sheet.worksheet('Buyer Details - 3 June')  # 0 index refers to the first sheet

# Read existing data from the sheet
existing_data = source_worksheet.get_all_values()
df = final_df

# Clear existing data from the sheet
if len(existing_data) > 0:
    source_worksheet.delete_rows(2, len(existing_data))
    print("Existing data cleared.")

# Write the entire DataFrame to the worksheet
data_to_append = [df.columns.tolist()] + df.values.tolist()
source_worksheet.insert_rows(data_to_append, 2)  # Insert after headers
print("Data written.")

print("Statuses updated.")


Existing data cleared.
Data written.
Statuses updated.


In [20]:
final_df.columns

Index(['SellerId', 'SellerName', 'BuyerId', 'BuyerName', 'BuyerPhone',
       'Recency Bucket'],
      dtype='object')

In [21]:
final_df.shape

(2421, 6)